## Imports

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import urllib.request
import os, struct
import numpy as np
import gzip
from array import array as pyarray
from numpy import append, array, int8, uint8, zeros
from collections import namedtuple
from sklearn.preprocessing import normalize

from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.cross_validation import KFold

## Load data

In [2]:
def download_data(directory='./data/'):
    """Download MNIST database"""
    urllib.request.urlretrieve("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz",
                               directory + "/train-images-idx3-ubyte.gz")
    urllib.request.urlretrieve("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz",
                               directory + "/train-labels-idx1-ubyte.gz")
    urllib.request.urlretrieve("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz",
                               directory + "/t10k-images-idx3-ubyte.gz")
    urllib.request.urlretrieve("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz",
                               directory + "/t10k-labels-idx1-ubyte.gz")

In [3]:
def load_mnist(dataset="training", digits=np.arange(10), path="./data/"):
    """Loads MNIST files into 3D numpy arrays

    Adapted from: http://abel.ee.ucla.edu/cvxopt/_downloads/mnist.py
    """

    if dataset == "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte.gz')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte.gz')
    elif dataset == "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte.gz')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte.gz')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    flbl = gzip.open(fname_lbl, 'rb')
    magic_nr, size = struct.unpack(">II", flbl.read(8))
    lbl = pyarray("b", flbl.read())
    flbl.close()

    fimg = gzip.open(fname_img, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", fimg.read(16))
    img = pyarray("B", fimg.read())
    fimg.close()

    ind = [k for k in range(size) if lbl[k] in digits]
    N = len(ind)

    images = zeros((N, rows, cols), dtype=uint8)
    labels = zeros((N, 1), dtype=int8)
    for i in range(len(ind)):
        images[i] = array(img[ ind[i]*rows*cols : (ind[i]+1)*rows*cols ]).reshape((rows, cols))
        labels[i] = lbl[ind[i]]

    return images, labels

In [4]:
def split_data():
    """Preprocess and return training and testing datasets"""

    images = namedtuple('MNIST_images', ['train', 'test'])
    labels = namedtuple('MNIST_labels', ['train', 'test'])
    images.train, labels.train = load_mnist('training')
    images.test, labels.test = load_mnist('testing')

    images.train = images.train.reshape(images.train.shape[0], -1)
    images.test = images.test.reshape(images.test.shape[0], -1)

    images.train = normalize(images.train.astype(np.float), axis=1)
    images.test = normalize(images.test.astype(np.float), axis=1)

    labels.train = labels.train.ravel()
    labels.test = labels.test.ravel()

    return images, labels

In [5]:
download_data()

In [6]:
images, labels = split_data()

## Stacking

In [7]:
def reverse_splits(splits):
    return [(split[1], split[0]) for split in splits]

In [8]:
class Stacking(object):    
    """Base class for stacking method of learning"""

    def __init__(self, base_fitter, meta_fitter, 
                 split=lambda I: list(KFold(n=I.size, n_folds=2, shuffle=True)),
                 decision_rule=lambda estimations: max(set(estimations), key=estimations.count)):
        self.split = split
        self.base_fitter = base_fitter
        self.meta_fitter = meta_fitter
        self.decision_rule = decision_rule  
    
    def fit(self, X, y):
        """Build compositions of classifiers.
        
        Parameters
        ----------
        X : array-like or sparse matrix of shape = [n_samples, n_features]
         
        y : array-like, shape = [n_samples]
        
        Returns
        -------
        self : object
            Returns self.
        """
        self.classifiers = []
        I = np.arange(y.size)
        partitions = self.split(I)

        for partition in partitions:
            base_subsample, meta_subsample = partition

            base_classifier = self.base_fitter(X[base_subsample], y[base_subsample])
                    
            X_meta = np.hstack((X[meta_subsample], 
                                base_classifier.predict(X[meta_subsample]).reshape(meta_subsample.size, -1)))
            
            meta_classifier = self.meta_fitter(X_meta, y[meta_subsample])
            
            self.classifiers.append(namedtuple('classifier', ['base', 'meta'])(base_classifier, meta_classifier))
            
        return self
    
    def predict(self, X):      
        """Predict class for X.

        The predicted class of an input sample is computed as the majority
        prediction of the meta-classifiers.
        
        Parameters
        ----------
        X : array-like or sparse matrix of shape = [n_samples, n_features]
        
        Returns
        -------
        y : array of shape = [n_samples]
            The predicted classes.
        """
        y_predicted = []
       
        estimations_matrix = np.array([], dtype=int).reshape(X.shape[0], 0)

        for classifier in self.classifiers:
            meta_features = classifier.base.predict(X)
            estimations_matrix = np.column_stack((estimations_matrix, 
                                            classifier.meta.predict(np.column_stack((X, meta_features)))))
            
        y_predicted = [self.decision_rule(list(estimations)) for estimations in estimations_matrix]

        return y_predicted     

In [9]:
class Classifier(object):
    """Classifier wrapper"""
    def __init__(self, predict_function):
        self.predict = predict_function

def get_SVM_fitter(C=1., kernel='linear', degree=3, gamma=1.):
    return lambda X, y: Classifier(SVC(C=C, kernel=kernel, degree=degree, gamma=gamma).fit(X, y).predict)

def SVM_fitter(X, y):
    classifier = SVC(kernel=kernel, degree=degree, gamma=gamma).fit(X, y)
    return Classifier(classifier.predict)

def logit_fitter(X, y):
    classifier = LogisticRegression('l2', False).fit(X, y)
    return Classifier(classifier.predict)

def random_forest_fitter(X, y):
    classifier = RandomForestClassifier().fit(X, y)
    return Classifier(classifier.predict)

def random_forest_proba_fitter(X, y):
    classifier = RandomForestClassifier().fit(X, y)
    return Classifier(classifier.predict_proba)

def extra_trees_proba_fitter(X, y):
    classifier = ExtraTreesClassifier().fit(X, y)
    return Classifier(classifier.predict_proba)

# Experiments

## Single classifier 

In [10]:
pca = PCA(50).fit(images.train)
pca_images_train = pca.transform(images.train)
pca_images_test = pca.transform(images.test)

In [11]:
models = namedtuple('models', ['logit', 'svm', 'random_forest', 'extra_trees'])
predictions = namedtuple('predictions', ['logit', 'svm', 'random_forest', 'extra_trees'])

### Logistic regression

In [12]:
models.logit = LogisticRegression('l2', False).fit(pca_images_train, labels.train)
predictions.logit = models.logit.predict(pca_images_test)

In [13]:
np.mean(predictions.logit != labels.test)

0.098100000000000007

### SVM

In [14]:
models.svm = SVC(kernel = 'linear').fit(pca_images_train, labels.train)
predictions.svm = models.svm.predict(pca_images_test)

In [15]:
np.mean(predictions.svm != labels.test)

0.062399999999999997

### Random forest classifier

In [16]:
models.random_forest = RandomForestClassifier().fit(pca_images_train, labels.train)
predictions.random_forest = models.random_forest.predict(pca_images_test)

In [17]:
np.mean(predictions.random_forest != labels.test)

0.069000000000000006

### Extra trees classifier

In [18]:
models.extra_trees = ExtraTreesClassifier().fit(pca_images_train, labels.train)
predictions.extra_trees = models.extra_trees.predict(pca_images_test)

In [19]:
np.mean(predictions.extra_trees != labels.test)

0.071999999999999995

## Stacking classifier

### Logistic regression + SVM

In [20]:
logit_svm = Stacking(base_fitter=logit_fitter, meta_fitter=get_SVM_fitter(C=5, kernel='poly', degree = 2), 
                     split=lambda I: list(KFold(n=I.size, n_folds=5, shuffle=True)))
logit_svm_predictions = logit_svm.fit(pca_images_train, labels.train).predict(pca_images_test)

In [21]:
np.mean(logit_svm_predictions != labels.test)

0.0378

### Logistic regression + Random forest

In [22]:
logit_rf = Stacking(base_fitter=logit_fitter, meta_fitter=random_forest_fitter, 
                    split=lambda I: list(KFold(n=I.size, n_folds=5, shuffle=True)))
logit_rf_predictions = logit_rf.fit(pca_images_train, labels.train).predict(pca_images_test)

In [23]:
np.mean(logit_rf_predictions != labels.test)

0.0746

### Extra trees + Logistic regression

In [24]:
et_logit = Stacking(base_fitter=extra_trees_proba_fitter, meta_fitter=logit_fitter, 
                    split=lambda I: list(KFold(n=I.size, n_folds=5, shuffle=True)))
et_logit_predictions = et_logit.fit(pca_images_train, labels.train).predict(pca_images_test)

In [25]:
np.mean(et_logit_predictions != labels.test)

0.041700000000000001

### SVM + SVM

In [26]:
svm_svm = Stacking(base_fitter=get_SVM_fitter(), meta_fitter=get_SVM_fitter(C=5, kernel='poly', degree = 2), 
                   split=lambda I: list(KFold(n=I.size, n_folds=5, shuffle=True)))
svm_svm_predictions = svm_svm.fit(pca_images_train, labels.train).predict(pca_images_test)

In [27]:
np.mean(svm_svm_predictions != labels.test)

0.038399999999999997

### Random forest + SVM

In [28]:
rf_svm = Stacking(base_fitter=random_forest_fitter, meta_fitter=get_SVM_fitter(C=10, kernel='poly', degree = 2), 
                  split=lambda I: list(KFold(n=I.size, n_folds=5, shuffle=True)))
rf_svm_predictions = rf_svm.fit(pca_images_train, labels.train).predict(pca_images_test)

In [29]:
np.mean(rf_svm_predictions != labels.test)

0.0339

### Extra trees + SVM

In [30]:
wildfowl = Stacking(base_fitter=extra_trees_proba_fitter, meta_fitter=get_SVM_fitter(C=5, kernel='poly', degree = 2), 
                    split=lambda I: list(KFold(n=I.size, n_folds=5, shuffle=True)))
wildfowl_predictions = wildfowl.fit(pca_images_train, labels.train).predict(pca_images_test)

In [31]:
np.mean(wildfowl_predictions != labels.test)

0.024199999999999999

In [35]:
for n_folds in [2, 3, 5, 10, 15, 20]:   
    et_logit = Stacking(base_fitter=extra_trees_proba_fitter, meta_fitter=get_SVM_fitter(C=5, kernel='poly', degree = 2), 
                        split=lambda I: list(KFold(n=I.size, n_folds=n_folds, shuffle=True)))
    et_logit_predictions = et_logit.fit(pca_images_train, labels.train).predict(pca_images_test)
    print(np.mean(et_logit_predictions != labels.test))

0.0237
0.0215
0.0249
0.0269
0.0275
0.0282
